In [1]:
# import all necessary packages & modules
import os
import time
import numpy as np
import pandas as pd
import json
import pyomo.environ as pyomo
from pyomo.opt import ProblemFormat
import matplotlib.pyplot as plt
import networkx as nx
import highspy
from networkx.readwrite import json_graph
from pyomo.opt import SolverStatus, TerminationCondition
from good_model_working import utils


import good_model_working
from good_model_working.reload import deep_reload
from good_model_working import opt_model
from good_model_working import diagnostics

In [2]:
deep_reload(good_model_working)

# Removing generators for testing
gen_to_remove = [
        #'Fossil Waste', 
        #'Municipal Solid Waste', 
        #'Non-Fossil Waste', 
        'Pumped Storage',
        'Fuel Cell',
        'Landfill Gas', 
        #"Energy Storage", 
        #"Solar PV", 
        #"Onshore Wind", 
        #'New Battery Storage', 
        #'IMPORT', 
        #'Tires',
        'Offshore Wind', 
        'Solar Thermal'
        ]


# import the model data & sets
input_sets = utils.get_sets('/Users/peterambiel/Desktop/good_model/Model Input/all_input_sets_sorted.json')
model_sets = utils.filter_sets(input_sets, gen_to_remove)

graph = utils.create_graph('/Users/peterambiel/Desktop/good_model/Model Input/all_input_objects.json')

In [3]:
deep_reload(good_model_working)

# MODEL TESTING
'''
    To analyze specific regions, input the appropriate subregion
    to the variable sub_region below
'''

user_input = input('Please select your region(s) for analysis (separated by commas): \nAll\nFlorida\nMISO\nNew England\nPJM\nSSP\nSoutheast\nERCOT\nWECC\n')

subgraph, sub_nodes = utils.get_subgraph(user_input, graph)

constraint_to_deactivate = ['generator', 'solar', 'wind', 'storage']

model_data = {
    'test_nodes': True,
    'test_cons': False,
    'graph': graph, 
    'sets': model_sets, 
    'subgraph_nodes': sub_nodes, 
    'subgraph': subgraph,
    'constraint_deactivation': None
}

In [4]:
# import good_model_working
deep_reload(good_model_working)

#Change this to point to your version of cbc or use another solver
solver_name='appsi_highs'

# run model 
problem = good_model_working.opt_model.Opt_Model(model_data, solver_name)

[+  54.85] Grid built
[+   0.01] Sets built
[+   4.73] Parameters built
[+   7.32] Variables built
[+  12.90] Objective Function built
[+ 143.26] Local Constraints built
[+   1.85] Transmission constraints built
[+   0.00] Starting balanacing constraint...
[+ 188.94] Balancing constraint built
[+   0.00] All Constraints built
[+   0.00] Model built


In [ ]:
deep_reload(good_model_working.utils)

# Generate simple model statistics
utils.get_model_statistcs(problem.model)
utils.get_total_generator_count(problem.model)

In [5]:
# Solve the model
problem.solve_model()

# print the objective value
print(f"Total system cost: ${pyomo.value(problem.model.obj_func):,.2f}")

Running HiGHS 1.7.0 (git hash: 27ccfaa): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [2e-07, 1e+00]
  Cost   [7e+00, 3e+03]
  Bound  [1e-08, 1e-08]
  RHS    [3e-02, 3e+06]
Presolving model
139529 rows, 6748996 cols, 12322972 nonzeros  4s
136743 rows, 2343854 cols, 7548307 nonzeros  10s
136743 rows, 2343854 cols, 7548307 nonzeros  12s
Presolve : Reductions: rows 136743(-8215761); columns 2343854(-5798410); elements 7548307(-19112447)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     1.2022304356e-02 Pr: 130381(2.89688e+07) 13s
      15440     3.0985623417e+08 Pr: 63835(2.52105e+09); Du: 0(6.73799e-07) 19s
      19924     3.8321653058e+08 Pr: 68447(6.67347e+08); Du: 0(7.70923e-07) 24s
      25558     4.2772721969e+08 Pr: 67902(7.49227e+09); Du: 0(9.05569e-07) 29s
      31189     4.4963202369e+08 Pr: 49404(1.88983e+09); Du: 0(1.13484e-06) 35s
      34092     4.66325692

In [6]:
deep_reload(good_model_working)

# Get model results
results = problem.get_results()

In [7]:
# get diagnostics
deep_reload(diagnostics)

hourly_mix = diagnostics.get_hourly_gen_mix(results)
annual_mix = diagnostics.get_annual_gen_mix(results)

In [8]:
# Compare 2021 US annual mix baseline to model generated fuel mix
deep_reload(diagnostics)

diagnostics.compare_annual_mix_to_baseline(annual_mix)

                        Resource Percentage_Baseline Percentage_Model
0                           Coal             21.900%           1.028%
1                            Oil              0.600%           0.000%
2                            Gas             38.400%           1.015%
3                   Other Fossil              0.500%           0.000%
4                        Nuclear             18.900%           8.589%
5                          Hydro              6.000%          41.852%
6                        Biomass              1.300%           0.121%
7                           Wind              9.200%           0.000%
8                          Solar              2.800%           0.004%
9                     Geothermal              0.400%          47.390%
10  Other unknown/purchased fuel              0.100%           0.000%


In [ ]:
# plot diagnostics
deep_reload(diagnostics)

diagnostics.plot_hourly_gen_mix(hourly_mix)